# Projecting operators to the single-excitation subspace

In this notebook, we develop a routine that project and/or construct quantum
operators in the
single-excitation subspace

In [1]:
% load_ext autoreload
% autoreload 2

In [ ]:
import itertools
import time
from collections import OrderedDict
from src.dicke_half_model_v2 import num_vals as num_vals_func, dicke_guess_controls
from src.algebra_v1 import split_hamiltonian
import qutip
import numpy as np

In [ ]:
T=5;theta=0; E0_cycles=2; mcwf=False; non_herm=False;
lambda_a=1.0; J_T_conv=1e-4; iter_stop=5000; nt=None;
max_ram_mb=100000; kappa=0.01; seed=None; observables='all';
keep_pulses='prev'; config='config'

In [ ]:
def qobj_extract_single_excitation(qobj):
    """Given a Qutip quantum object (operator or state), that is defined in the
    full N-node chain Hilbert space, return a new quantum object in the
    zero/single-excitation subspace.

    The input quantum object must have the tensor structure (TLS_q ⊗ TLS_c)^N,
    as reflected in the `dims` property.
    """
    assert qobj.dims[0] == [2 for _ in qobj.dims[0]]
    SINGLE_EXCITATION = 1
    subspace_levels = [
        i for (i, nums)  # nums = tuple of quantum numbers in the full basis
        in enumerate(itertools.product(*[range(n) for n in qobj.dims[0]]))
        if sum(nums) <= SINGLE_EXCITATION]
    return qobj.extract_states(subspace_levels)

In [ ]:
def symbolic_hamiltonians_v1(n_nodes):
    from src.single_sided_network_v1 import network_slh as network_slh_func_v1

    slh = network_slh_func_v1(n_cavity=2, n_nodes=n_nodes, topology='open')
    controls = dicke_guess_controls(
        slh=slh, theta=theta, T=T, E0_cycles=E0_cycles, nt=nt, kappa=kappa)
    control_syms = list(controls.keys())
    ham_parts = split_hamiltonian(slh.H, use_cc=False, controls=control_syms)
    return ham_parts

In [ ]:
def symbolic_hamiltonians_v2(n_nodes):
    from src.single_sided_network_v2 import network_slh as network_slh_func_v2

    slh = network_slh_func_v2(n_cavity=2, n_nodes=n_nodes, topology='open')
    controls = dicke_guess_controls(
        slh=slh, theta=theta, T=T, E0_cycles=E0_cycles, nt=nt, kappa=kappa)
    control_syms = list(controls.keys())
    ham_parts = split_hamiltonian(slh.H, use_cc=False, controls=control_syms)
    return ham_parts

In [ ]:
def symbolic_lindblads(n_nodes):
    from src.single_sided_network_v2 import network_slh as network_slh_func_v2

    slh = network_slh_func_v2(n_cavity=2, n_nodes=n_nodes, topology='open')
    return slh.Ls

In [ ]:
def num_hamiltonians(n_nodes):

    from src.single_sided_network_v1 import network_slh as network_slh_func_v2
    # assuming symbolic_hamiltonians_v2 is tested, there's no different between
    # a potential "v1" and "v2"

    slh = network_slh_func_v2(n_cavity=2, n_nodes=n_nodes, topology='open')
    num_vals = num_vals_func(theta=theta, n_nodes=n_nodes, kappa=kappa)
    controls = dicke_guess_controls(
        slh=slh, theta=theta, T=T, E0_cycles=E0_cycles, nt=nt, kappa=kappa)
    H_num = slh.H.substitute(num_vals)
    control_syms = list(controls.keys())
    ham_parts = split_hamiltonian(H_num, use_cc=False, controls=control_syms)
    num_hams = OrderedDict()
    for (key, H) in ham_parts.items():
        num_hams[key] = H.substitute({sym: 1 for sym in control_syms})
    return num_hams

In [ ]:
def num_lindblads(n_nodes):
    num_vals = num_vals_func(theta=theta, n_nodes=n_nodes, kappa=kappa)
    Ls = symbolic_lindblads(n_nodes)
    return [L.substitute(num_vals) for L in Ls]

In [ ]:
def qutip_hamiltonians_v1(n_nodes):
    """Construct the Hamiltonians by constructing them in the full space, and
    then converting them to the single-excitation subspace"""

    from src.single_sided_network_v1 import network_slh as network_slh_func_v1
    from qnet.convert.to_qutip import convert_to_qutip

    slh = network_slh_func_v1(n_cavity=2, n_nodes=n_nodes, topology='open')
    num_vals = num_vals_func(theta=theta, n_nodes=n_nodes, kappa=kappa)
    controls = dicke_guess_controls(
        slh=slh, theta=theta, T=T, E0_cycles=E0_cycles, nt=nt, kappa=kappa)
    H_num = slh.H.substitute(num_vals)
    control_syms = list(controls.keys())
    ham_parts = split_hamiltonian(H_num, use_cc=False, controls=control_syms)
    hs = H_num.space
    for (key, H) in ham_parts.items():
        H = H.substitute({sym: 1 for sym in control_syms})
        ham_parts[key] = qobj_extract_single_excitation(
                convert_to_qutip(H, full_space=hs))
    return ham_parts

In [ ]:
def qutip_lindblads_v1(n_nodes):
    """Construct the Lindblad ops by constructing them in the full space, and
    then converting them to the single-excitation subspace"""

    from src.single_sided_network_v1 import network_slh as network_slh_func_v1
    from qnet.convert.to_qutip import convert_to_qutip

    slh = network_slh_func_v1(n_cavity=2, n_nodes=n_nodes, topology='open')
    num_vals = num_vals_func(theta=theta, n_nodes=n_nodes, kappa=kappa)
    hs = slh.H.space
    res = []
    for L in slh.Ls:
        L_num = L.substitute(num_vals)
        res.append(qobj_extract_single_excitation(
            convert_to_qutip(L_num, full_space=hs)))
    return res

In [ ]:
def get_spin_index(n_nodes):
    """Return the spin_index for the given number of nodes (for
    debugging/inspection)"""
    from src.single_sided_network_v2 import network_slh as network_slh_func_v2
    from src.to_single_excitation_qutip import construct_bit_index
    slh = network_slh_func_v2(n_cavity=2, n_nodes=n_nodes, topology='open')
    return construct_bit_index(slh.H.space)

In [ ]:
def qutip_hamiltonians_v2(n_nodes):
    """Construct the Hamiltonians directly in the single-excitation subspace"""

    from src.single_sided_network_v2 import network_slh as network_slh_func_v2
    from src.to_single_excitation_qutip import (
            convert_to_single_excitation_qutip, construct_bit_index)
    from qnet.algebra import pattern, wc, LocalSigma, Create, Destroy

    slh = network_slh_func_v2(n_cavity=2, n_nodes=n_nodes, topology='open')
    num_vals = num_vals_func(theta=theta, n_nodes=n_nodes, kappa=kappa)
    controls = dicke_guess_controls(
        slh=slh, theta=theta, T=T, E0_cycles=E0_cycles, nt=nt, kappa=kappa)
    H_num = slh.H.substitute(num_vals)
    control_syms = list(controls.keys())
    ham_parts = split_hamiltonian(H_num, use_cc=False, controls=control_syms)
    hs = H_num.space
    bit_index = construct_bit_index(hs)

    pat_localsigma = pattern(LocalSigma, wc('i'), wc('j'), hs=wc('hs'))

    def localsigma_to_create_destroy(i, j, hs):
        if (i, j) == ('g', 'e'):
            return Destroy(hs=hs)
        elif (i, j) == ('e', 'g'):
            return Create(hs=hs)
        else:
            raise ValueError("Invalid (i, j) = %s" % (str((i, j))))

    for (key, H) in ham_parts.items():
        H = H.substitute({sym: 1 for sym in control_syms})
        H = H.simplify([(pat_localsigma, localsigma_to_create_destroy)])
        ham_parts[key] = convert_to_single_excitation_qutip(
            H, bit_index, full_space=hs)
    return ham_parts

In [ ]:
def qutip_lindblads_v2(n_nodes):
    """Construct the Lindblad ops by constructing them in the full space, and
    then converting them to the single-excitation subspace"""

    from src.single_sided_network_v2 import network_slh as network_slh_func_v2
    from src.to_single_excitation_qutip import (
            convert_to_single_excitation_qutip, construct_bit_index)

    slh = network_slh_func_v2(n_cavity=2, n_nodes=n_nodes, topology='open')
    num_vals = num_vals_func(theta=theta, n_nodes=n_nodes, kappa=kappa)
    hs = slh.H.space
    bit_index = construct_bit_index(hs)
    res = []
    for L in slh.Ls:
        L_num = L.substitute(num_vals)
        res.append(convert_to_single_excitation_qutip(
            L_num, bit_index, full_space=hs))
    return res

In [ ]:
def qutip_states_v1(n_nodes):
    """Construct the states in the full space, and
    then converting them to the single-excitation subspace"""

    from src.single_sided_network_v1 import network_slh as network_slh_func_v1
    from src.qdyn_model_v2 import dicke_state, logical_2q_state

    slh = network_slh_func_v1(n_cavity=2, n_nodes=n_nodes, topology='open')
    hs = slh.H.space
    dicke1 = qobj_extract_single_excitation(dicke_state(hs, excitations=1))
    psi10 = qobj_extract_single_excitation(logical_2q_state(hs, 1, 0))
    return {'dicke1': dicke1, 'psi10': psi10}

In [ ]:
def _state_to_fmt(qutip_state, fmt):
    import QDYN.linalg
    if fmt == 'qutip':
        return qutip_state
    elif fmt == 'numpy':
        return QDYN.linalg.vectorize(qutip_state.data.todense())
    else:
        raise ValueError("Unknown fmt")

In [ ]:
def single_excitation_dicke_state(bit_index, fmt='qutip'):
    """Return the zero/single-excitation dicke state for the given system,
    directly using the single-exctiation subspace encoding. Assumes
    that the Hilbert spaces for the qubits at the different nodes are labeled
    'q1', 'q2', ...

    For example for a Hilbert space ``('q1', 'c1', 'q2', 'c2', 'q3', 'c3')``,
    the Dicke state is ``(|000010> + |001000> + |100000>) / sqrt(3)``. The
    corresponding bit-index is

        {'000000': 0,
         '000001': 1,
         '000010': 2,
         '000100': 3,
         '001000': 4,
         '010000': 5,
         '100000': 6}

    and only the eigenstates 2, 4, and 6 contribute to the Dicke state.
    """
    N = len(bit_index)
    terms = [qutip.basis(N, i) for i in range(2, N, 2)]
    res = sum(terms) / np.sqrt(len(terms))
    return _state_to_fmt(res, fmt)

In [ ]:
def single_excitation_initial_state(bit_index, fmt='qutip'):
    """Return the initial state "e0....0" directly using the single-excitation
    subspace enconding"""
    N = len(bit_index)
    i = bit_index["1" + "0" * (N-2)]
    assert i == N - 1
    return _state_to_fmt(qutip.basis(N, i), fmt)

In [ ]:
def qutip_states_v2(n_nodes):
    """Construct the states in the full space, and
    then converting them to the single-excitation subspace"""

    from src.single_sided_network_v2 import network_slh as network_slh_func_v2
    from src.to_single_excitation_qutip import construct_bit_index

    slh = network_slh_func_v2(n_cavity=2, n_nodes=n_nodes, topology='open')
    hs = slh.H.space
    bit_index = construct_bit_index(hs)
    dicke1 = single_excitation_dicke_state(bit_index)
    psi10 = single_excitation_initial_state(bit_index)
    return {'dicke1': dicke1, 'psi10': psi10}

In [ ]:
def test_symbolic_hamiltonians(n_nodes):
    t1 = time.time()
    hams_v1 = symbolic_hamiltonians_v1(n_nodes)
    t2 = time.time()
    hams_v2 = symbolic_hamiltonians_v2(n_nodes)
    t3 = time.time()
    print("v1: %.2f seconds" % (t2-t1))
    print("v2: %.2f seconds" % (t3-t2))
    assert hams_v1.keys() == hams_v2.keys()
    for key in hams_v1:
        assert hams_v1[key] == hams_v2[key]
    print("OK")

In [ ]:
def test_qutip_hamiltonians(n_nodes):
    t1 = time.time()
    hams_v1 = qutip_hamiltonians_v1(n_nodes)
    t2 = time.time()
    hams_v2 = qutip_hamiltonians_v2(n_nodes)
    t3 = time.time()
    print("v1: %.2f seconds" % (t2-t1))
    print("v2: %.2f seconds" % (t3-t2))
    assert hams_v1.keys() == hams_v2.keys()
    for key in hams_v1:
        assert hams_v1[key] == hams_v2[key], "Ham %s does not match" % key
    print("OK")

In [ ]:
def test_qutip_states(n_nodes):
    t1 = time.time()
    states_v1 = qutip_states_v1(n_nodes)
    t2 = time.time()
    states_v2 = qutip_states_v2(n_nodes)
    t3 = time.time()
    print("v1: %.2f seconds" % (t2-t1))
    print("v2: %.2f seconds" % (t3-t2))
    assert states_v1.keys() == states_v2.keys()
    for key in states_v1:
        assert states_v1[key] == states_v2[key], \
            "state %s does not match" % key
    print("OK")

In [ ]:
def test_qutip_lindblads(n_nodes):
    t1 = time.time()
    Ls_v1 = qutip_lindblads_v1(n_nodes)
    t2 = time.time()
    Ls_v2 = qutip_lindblads_v2(n_nodes)
    t3 = time.time()
    print("v1: %.2f seconds" % (t2-t1))
    print("v2: %.2f seconds" % (t3-t2))
    for i, L in enumerate(Ls_v1):
        assert L.norm('max') > 0.14
        assert L == Ls_v2[i], "L does not match"
    print("OK")

In [ ]:
test_symbolic_hamiltonians(4)
test_qutip_hamiltonians(4)
test_qutip_states(4)
test_qutip_lindblads(4)